In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path
import pickle

In [16]:
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.screening_modes_api import ScreeningModesApi

from exact_sync.v1.models import ImageSet, Image, ScreeningMode
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [31]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = 'imagetagger'
configuration.host =  "https://exact.cs.fau.de/"

image_set_ids = [14]

In [32]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
images_api = ImagesApi(client)
screening_api = ScreeningModesApi(client)

In [33]:
images = {}
screening_modes = {}

for image_set in image_set_ids:
    for image in images_api.list_images(pagination=False, image_set=image_set, image_type=0).results:
        images[image.name] = image
        for screening in screening_api.list_screening_modes(image=image.id).results:
            screening_modes[image.name] = screening

In [34]:
data = {}

for image_name, screening in screening_modes.items():
    
    data[image_name] = {
        "screening_tiles": screening.screening_tiles,
        "x_resolution": screening.x_resolution,
        "y_resolution": screening.y_resolution,
        
        "x_steps": screening.x_steps,
        "y_steps": screening.y_steps,
    }

with open('Screening.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
data = []

for image_name in screening_modes:
    
    screening = screening_modes[image_name]
    screened_tiles = [tiles for tiles in screening.screening_tiles.values() if tiles["Screened"] == True]
    all_tiles = [tiles for tiles in screening.screening_tiles.values()]
    
    data.append([image_name, len(screened_tiles), len(all_tiles), len(screened_tiles) / len(all_tiles) * 100])
data = pd.DataFrame(data, columns=["image", "screened", "unscreened", "%"])
data

,image,screened,unscreened,%
0,BAL 1 Spray 2.svs,106,4212,2.516619
1,BAL Promyk Spray 4.svs,719,726,99.035813
2,BAL AIA Blickfang Luft.svs,696,704,98.863636
3,BAL Booker Spray 3.svs,277,3588,7.720178
4,BAL Bubi Spray 1.svs,622,3128,19.884910
5,BAL cent blue Luft 2.svs,20,3128,0.639386


In [36]:
x = 12038
y = 5505

In [37]:
screening = screening_modes["BAL cent blue Luft 2.svs"]
tiles = screening.screening_tiles
screening

{'current_index': 1619,
 'id': 485,
 'image': 225,
 'screening_tiles': {'0': {'Screened': False,
                           'x_max': 667,
                           'x_min': 0,
                           'y_max': 449,
                           'y_min': 0},
                     '1': {'Screened': False,
                           'x_max': 1334,
                           'x_min': 667,
                           'y_max': 449,
                           'y_min': 0},
                     '10': {'Screened': False,
                            'x_max': 7337,
                            'x_min': 6670,
                            'y_max': 449,
                            'y_min': 0},
                     '100': {'Screened': False,
                             'x_max': 6003,
                             'x_min': 5336,
                             'y_max': 1347,
                             'y_min': 898},
                     '1000': {'Screened': False,
                              'x_max': 2334

In [38]:
tiles["1571"]

{'x_max': 5336,
 'x_min': 4669,
 'y_max': 15715,
 'y_min': 15266,
 'Screened': False}

In [11]:
x_step = int(np.floor(x / screening.x_resolution))
y_step = int(np.floor(y / screening.y_resolution))
(x_step, y_step)

(18, 12)

In [12]:
tiles[str((y_step * screening.x_steps) + x_step)]

{'x_max': 12673,
 'x_min': 12006,
 'y_max': 5837,
 'y_min': 5388,
 'Screened': True}

In [60]:
def filterAnnotations(image_name, xmin, ymin, screening_modes, patch_x:int = 1024, patch_y:int = 1024):
    
    screening = screening_modes[image_name]
    tiles = screening.screening_tiles    
    
    xmin, ymin = max(1, int(xmin - patch_x / 2)), max(1, int(ymin - patch_y / 2))
    
    x_step = int(np.floor(xmin / screening.x_resolution))
    y_step = int(np.floor(ymin / screening.y_resolution))
    
    return tiles[str((y_step * screening.x_steps) + x_step)]['Screened']

In [53]:
annotations_path = Path("Asthma_Annotations.pkl")
annotations = pd.read_pickle(annotations_path)

annotations["center_x"] = [vector["x1"] + ((vector["x2"] - vector["x1"]) / 2) for vector in annotations["vector"]]
annotations["center_y"] = [vector["y1"] + ((vector["y2"] - vector["y1"]) / 2) for vector in annotations["vector"]]

annotations

,id,image_id,image_name,image_set,class,vector,unique_identifier,user_id,deleted,last_editor,center_x,center_y
0,172441,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13612, 'x2': 13666, 'y1': 12371, 'y2': ...",a268b46f-e13f-480e-b96d-c7900790ad5f,10,False,None,13639.0,12398.0
1,172442,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13562, 'x2': 13618, 'y1': 12437, 'y2': ...",e8d67efd-0baf-4f78-ad73-b8f77422b826,10,False,None,13590.0,12465.0
2,172443,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13482, 'x2': 13546, 'y1': 12458, 'y2': ...",55aa3d51-7f30-43a0-83ef-d9acc74d9a21,10,False,None,13514.0,12490.0
3,172444,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13471, 'x2': 13533, 'y1': 12281, 'y2': ...",8f259e8f-119b-44de-8a90-2c53e7ee0e60,10,False,None,13502.0,12312.0
4,172446,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13591, 'x2': 13673, 'y1': 12204, 'y2': ...",052b3f5a-f6f6-4b0d-a134-9f4797601dc4,10,False,None,13632.0,12245.0
...,...,...,...,...,...,...,...,...,...,...,...,...
92614,301669,225,BAL cent blue Luft 2.svs,14,ROI,"{'x1': 14171, 'x2': 14987, 'y1': 14271, 'y2': ...",b368f8cc-0ab1-46a0-b157-e032dc8ec9ad,12,False,None,14579.0,14506.5
92615,301670,225,BAL cent blue Luft 2.svs,14,ROI,"{'x1': 10914, 'x2': 11318, 'y1': 16710, 'y2': ...",db31d707-509d-405b-a102-1ff50fb9941b,12,True,None,11116.0,16918.0
92616,301671,225,BAL cent blue Luft 2.svs,14,ROI,"{'x1': 8846, 'x2': 9351, 'y1': 16794, 'y2': 17...",4900171f-061b-4bb4-a6f1-599db8f15587,12,False,None,9098.5,17022.0
92617,301672,225,BAL cent blue Luft 2.svs,14,ROI,"{'x1': 9747, 'x2': 10201, 'y1': 16358, 'y2': 1...",4999a017-6406-4b68-b337-31f32983e4f5,12,True,None,9974.0,16556.5


In [61]:
annotations[annotations.apply(lambda x: filterAnnotations(x["image_name"], x["vector"]["x1"], x["vector"]["y1"], screening_modes), axis=1)]

,id,image_id,image_name,image_set,class,vector,unique_identifier,user_id,deleted,last_editor,center_x,center_y
0,172441,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13612, 'x2': 13666, 'y1': 12371, 'y2': ...",a268b46f-e13f-480e-b96d-c7900790ad5f,10,False,None,13639.0,12398.0
1,172442,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13562, 'x2': 13618, 'y1': 12437, 'y2': ...",e8d67efd-0baf-4f78-ad73-b8f77422b826,10,False,None,13590.0,12465.0
2,172443,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13482, 'x2': 13546, 'y1': 12458, 'y2': ...",55aa3d51-7f30-43a0-83ef-d9acc74d9a21,10,False,None,13514.0,12490.0
3,172444,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13471, 'x2': 13533, 'y1': 12281, 'y2': ...",8f259e8f-119b-44de-8a90-2c53e7ee0e60,10,False,None,13502.0,12312.0
4,172446,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13591, 'x2': 13673, 'y1': 12204, 'y2': ...",052b3f5a-f6f6-4b0d-a134-9f4797601dc4,10,False,None,13632.0,12245.0
...,...,...,...,...,...,...,...,...,...,...,...,...
92599,140446,225,BAL cent blue Luft 2.svs,14,Mehrkernige,"{'x1': 9537, 'x2': 9653, 'y1': 15485, 'y2': 15...",38e90e33-417f-43b1-89c6-56969d03f4dc,10,False,None,9595.0,15543.0
92602,140680,225,BAL cent blue Luft 2.svs,14,Mehrkernige,"{'x1': 5070, 'x2': 5150, 'y1': 15797, 'y2': 15...",07cce53f-ff05-40fa-8a44-6f4821abb72d,10,False,None,5110.0,15837.0
92603,140760,225,BAL cent blue Luft 2.svs,14,Mehrkernige,"{'x1': 4274, 'x2': 4376, 'y1': 16446, 'y2': 16...",bcafcba4-f1a7-4e73-8822-24a82449af2f,10,False,None,4325.0,16497.0
92604,140805,225,BAL cent blue Luft 2.svs,14,Mehrkernige,"{'x1': 4795, 'x2': 4903, 'y1': 16364, 'y2': 16...",ea69207c-0a9e-4af5-8ed6-73c9220ec5ae,10,False,None,4849.0,16418.0
